In [7]:
import pandas as pd
import requests
import re
import os
import random
import numpy as np
from tensorflow.keras.models import *
from keras import callbacks
from tensorflow.keras.layers import *
from keras.layers import *
from PIL import Image
import csv
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle
import torch
import torchvision
from torchvision import models
import torch.nn as nn
import matplotlib.pyplot as plt
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from sklearn.preprocessing import LabelEncoder
import torch.nn.functional as F
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from torchvision import models
from torch.utils.data import Dataset, DataLoader

In [8]:
def segment_activities(df_s):
    activitiesSeq = []
    ponentialIndex = df_s.label.ne(df_s.label.shift())
    ii = np.where(ponentialIndex == True)[0]
    for i_s, end in enumerate(ii):
        if i_s > 0:
            df_stmp = df_s[ii[i_s - 1]:end]
            activitiesSeq.append(df_stmp)
    return activitiesSeq

def sequences_to_sentences(activity_sequences_s):
    sentences_s = []
    label_sentences_s = []
    for i_s in range(len(activity_sequences_s)):
        sentence = generate_sentence(activity_sequences_s[i_s])
        sentences_s.append(sentence)
        label_sentences_s.append(activity_sequences_s[i_s].label.values[0])
    return sentences_s, label_sentences_s

def generate_sentence(df2):
    sentence = ""
    sensors = df2.sensor.values
#    values = df2.status.values
    for i_s in range(len(sensors)):
#        val = values[i_s]
        if i_s == len(sensors) - 1:
            sentence += "{}".format(sensors[i_s])
        else:
            sentence += "{} ".format(sensors[i_s])
    return sentence

def sliding_window(sequence, win_size_s, step_s=1):
    try:
        iter(sequence)
    except TypeError:
        raise Exception("**ERROR** sequence must be iterable.")
    numOfChunks = int(((len(sequence) - win_size_s) / step_s) + 1)

    if win_size_s > len(sequence):
        yield sequence[0:len(sequence)]
    else:
        for i_s in range(0, numOfChunks * step_s, step_s):
            yield sequence[i_s:i_s + win_size_s]

# 定义一个函数来填充1和0.5之间的值
def fill_between_values(col, df, col_name):
    first_one_index = col[col == 1].index.min()
    # 如果存在1的值，将其之前的所有值填充为0.5
    if first_one_index is not None:
        df.loc[:first_one_index - 1, col_name] = 0.5

    start_index = None
    previous_value = None
    for idx, value in col.items():
        if value == 1 or value == 0.5:
            if start_index is not None:
                if previous_value == 1:
                    df.loc[start_index:idx-1, col_name] = 1
                elif previous_value == 0.5:
                    df.loc[start_index:idx-1, col_name] = 0.5
                start_index = None
            previous_value = value
        elif start_index is None:
            start_index = idx

    return df[col_name]

def count_directories(path):
    dir_count = 0
    for root, dirs, files in os.walk(path):
        # 在第一层停止，不遍历子目录
        dir_count += len(dirs)
        break  # 只处理顶层目录，不进入子目录
    return dir_count

In [16]:
df = pd.read_csv('1013_clean.csv', header=None, names=["date", "time", "sensor", "status"])
df = df.iloc[::-1].reset_index(drop=True)

print(type(df))
print(df[:10])

# 将 'sensor' 列转换为整数
df = df[df['sensor'] != 'sensor']
df['sensor'] = pd.to_numeric(df['sensor'])
df['status'] = df['status'].astype(float)

# 定义特殊的sensor值
special_sensors = [100,101,102,103,104,105,106]

# 对每个特殊的sensor进行处理
for sensor in special_sensors:
    # 新增一列，并为符合条件的行填充对应的值
    df[sensor] = (df["sensor"] == sensor).astype(int) * df["status"]

for col_name in df.columns[4:]:
    df[col_name] = fill_between_values(df[col_name], df, col_name)
print(df[:10])


for sensor in special_sensors:
        # 删除包含特殊sensor值的行
    df = df[df["sensor"] != sensor]

print(df[:10])

df['label'] = df[special_sensors].apply(tuple, axis=1)
df.drop(columns=special_sensors, inplace=True)

print(df[:10])

# 获取 'status' 和 'sensor' 列
# 删除包含错误值的行
df = df[df['sensor'] != 'sensor']
df['sensor'] = pd.to_numeric(df['sensor'])
sensor_column = df['sensor'].astype(float)
status_column = df['status'].astype(float)

# 将 'status' 列中所有1变成0, 所有0.5变成-0.1
status_column[status_column == 1] = 0
status_column[status_column == 0.5] = -0.1

# 将处理后的 'status' 列和 'sensor' 列相加，并将结果赋值给 'sensor' 列
df['sensor'] = status_column + sensor_column

# 输出结果
print(df[:10])

<class 'pandas.core.frame.DataFrame'>
         date      time sensor status
0  2023/10/13  14:55:03      1    1.0
1  2023/10/13  14:55:04    100    1.0
2  2023/10/13  14:55:09      2    1.0
3  2023/10/13  14:55:10      3    1.0
4  2023/10/13  14:55:11     61    1.0
5  2023/10/13  14:55:11      1    0.5
6  2023/10/13  14:55:11    102    1.0
7  2023/10/13  14:55:12    101    1.0
8  2023/10/13  14:55:13     61    0.5
9  2023/10/13  14:55:13      2    0.5
         date      time  sensor  status  100  101  102  103  104  105  106
0  2023/10/13  14:55:03       1     1.0  0.5  0.5  0.5  0.5  0.5  0.5  0.5
1  2023/10/13  14:55:04     100     1.0  1.0  0.5  0.5  0.5  0.5  0.5  0.5
2  2023/10/13  14:55:09       2     1.0  1.0  0.5  0.5  0.5  0.5  0.5  0.5
3  2023/10/13  14:55:10       3     1.0  1.0  0.5  0.5  0.5  0.5  0.5  0.5
4  2023/10/13  14:55:11      61     1.0  1.0  0.5  0.5  0.5  0.5  0.5  0.5
5  2023/10/13  14:55:11       1     0.5  1.0  0.5  0.5  0.5  0.5  0.5  0.5
6  2023/10/13  14:5

In [17]:
#  Segment dataset in sequence of activity ##
print("STEP 1: segment dataset in sequence of activity")

#  Segment dataset in sequence of activity ##
print("STEP 3: segment dataset in sequence of activity")
activity_sequences = segment_activities(df)
print(activity_sequences[121:125])

#  Transform sequences of activity in sentences ##
print("STEP 4: transform sequences of activity in sentences")
sentences, label_sentences = sequences_to_sentences(activity_sequences)
print(sentences[:10])
print(label_sentences[:10])

#  Indexization ##
print("STEP 5: sentences indexization")
# 转换文本为整数序列
sequences = [list(map(float, s.split())) for s in sentences]
#sequences = [np.array(seq) for seq in sentences]

print(sequences[:5])

#  Split in sliding windows ##
print("STEP 6: split indexed sentences in sliding windows")
X_windowed = []
Y_windowed = []
X_windowed_sen = []
Y_windowed_sen = []
win_size = 8
step = 1
for i, s in enumerate(sequences):
    chunks = sliding_window(s, win_size, step)
    for chunk in chunks:
        X_windowed.append(chunk)
        Y_windowed.append(label_sentences[i])
print(X_windowed[0:10])
print(Y_windowed[0:10])

#  Pad windows ##
print("STEP 7: pad sliding windows")
#padded_windows = pad_sequences(X_windowed, padding ='post', value=0.0)

# 获取最大序列长度
max_length = max(len(seq) for seq in X_windowed)

# 使用 np.pad 函数填充每个序列
padded_windows = [np.pad(seq, (0, max_length - len(seq)), mode='constant', constant_values=0.0) for seq in X_windowed]

# 将结果转换为 NumPy 数组
padded_windows = np.array(padded_windows)

Y_windowed = np.array(Y_windowed)
Y_windowed = Y_windowed.astype(int)
print(padded_windows[:10])
print(Y_windowed[:10])
print(type(Y_windowed))
print(len(Y_windowed))
#  Save files ##
print("STEP 8: save sliding windows and labels")
# np.save("{}_{}_padded_x_step1_state_linyu.npy".format(r'/Users/zehaokou/Desktop/Technion/AI/plot/Linyu_dataset/', win_size), padded_windows)
# np.save("label_sentences.npy".format(r'/Users/zehaokou/Desktop/Technion/AI/plot/Linyu_dataset', win_size), Y_windowed)
# pickle_file_path = '/Users/zehaokou/Desktop/Technion/AI/plot/Linyu_dataset/indexed_sentences.pkl'
# with open(pickle_file_path, 'wb') as pickle_file:
#     pickle.dump(indexed_sentences, pickle_file)

STEP 1: segment dataset in sequence of activity
STEP 3: segment dataset in sequence of activity
[           date      time  sensor  status                                label
876  2023/10/13  15:35:48     0.9     0.5  (1.0, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5),            date      time  sensor  status                                label
878  2023/10/13  15:36:04     1.0     1.0  (0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5),            date      time  sensor  status                                label
880  2023/10/13  15:36:10     2.0     1.0  (1.0, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5)
881  2023/10/13  15:36:11     3.0     1.0  (1.0, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5)
882  2023/10/13  15:36:12     0.9     0.5  (1.0, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5)
883  2023/10/13  15:36:12    61.0     1.0  (1.0, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5),            date      time  sensor  status                                label
885  2023/10/13  15:36:14     1.9     0.5  (1.0, 0.5, 1.0, 0.5, 0.5, 0.5, 0.5)
886  2023/10/13  15:36:14    60

In [20]:
# 初始化一个空的图像数据列表和标签列表
image_data = []
labels = []

for i in range(len(padded_windows)):
    
    label = Y_windowed[i]
#    label = label.astype(str)
    label = np.array2string(label, precision=2, separator=',', suppress_small=True)
    labels.append(label)
    
label_encoder = LabelEncoder()
int_labels = label_encoder.fit_transform(labels)

In [22]:
for i in range(len(padded_windows)):
    
    label = int_labels[i]
    #label = label.astype(str)
    label = np.array2string(label, precision=2, separator=',', suppress_small=True)
    
#    label_dir = f"/Users/zehaokou/Desktop/Technion/AI/plot/Linyu_dataset/1013/plot_train_state_7/{label}"
    label_dir = f"images/plot_train_state_0/{label}"
    
    zero = np.zeros((100,100))
    for j in range(6):
        index = 99 - padded_windows[i][j]
        if index != 0:
            if index - index.astype(int) != 0:
                index = index.astype(int)
                zero[index][2*j:(2*j+1)] = 0.5
            elif index - index.astype(int) == 0:
                index = index.astype(int)
                zero[index][2*j:(2*j+1)] = 1
            else:
                zero[index][2*j:(2*j+1)] = 0

#############填充状态s    
    for row in range(100):
        # 找到当前纵坐标下所有值不为0的点的横坐标
        cols_with_value = [col for col in range(100) if zero[row][col] > 0]

        if len(cols_with_value) > 0:
            # 如果第一个值是0.5，从y=0向这一点连线
            if zero[row][cols_with_value[0]] == 0.5:
                zero[row][0:cols_with_value[0]] = 1

            # 对相邻的on和off之间进行填充
            for k in range(0, len(cols_with_value) - 1):
                # If the current point is 'on' and the next point is 'off', fill between them
                if zero[row][cols_with_value[k]] == 1 and zero[row][cols_with_value[k + 1]] == 0.5:
                    zero[row][cols_with_value[k]:cols_with_value[k + 1]] = 1

            # 如果最后一个值为1，从这点向y=not_nan_count*2 - 1连线
            if zero[row][cols_with_value[-1]] == 1:
                zero[row][cols_with_value[-1]:2*win_size-1] = 1

    for row in range(100):
# 找到当前纵坐标下所有值为1的点的横坐标
        cols_with_value = [col for col in range(100) if zero[row][col] == 1]

        # 将除了值为1的点外的其他点设置为0
        zero[row, :] = 0
        for col in cols_with_value:
            zero[row, col] = 1
        ###########

    # 将zero数组的值乘以255
    data = zero * 255
    # 将data数组的数据类型转换为'uint8'
    data = data.astype('uint8')
    # 确保保存图像的目录存在，如果不存在就创建
    if not os.path.exists(label_dir):
        os.makedirs(label_dir)
    
    # 生成图片文件的完整路径，这里使用一个计数器以防止覆盖同一标签下的不同图片
#    img_path = os.path.join(label_dir, f"{label}_{i}.png")
    img_path = os.path.join(label_dir, f"{label}_{i}.png")
    
    # 将图像数据保存为图片
    img = Image.fromarray(data)
    img.save(img_path)
    
    # 重置zero数组为224x224的零数组，为下一次迭代做准备
    zero = np.zeros((100, 100))


print('done')
print(labels[:10])
print(image_data[:10])
#labels = labels.cpu().numpy()
labels = np.array(labels)
print(type(labels))
print(type(image_data))

#labels = np.array(labels)
label_encoder = LabelEncoder()
int_labels = label_encoder.fit_transform(labels)
mapping = dict(zip(range(len(label_encoder.classes_)), label_encoder.classes_))
print(mapping)

done
['[0,0,0,0,0,0,0]', '[1,0,0,0,0,0,0]', '[1,1,1,0,0,0,0]', '[0,1,1,0,0,0,0]', '[1,1,1,0,0,0,0]', '[1,1,1,1,0,0,0]', '[1,1,1,1,0,0,0]', '[1,1,1,1,0,0,0]', '[1,1,1,1,0,0,0]', '[1,1,1,1,0,0,0]']
[]
<class 'numpy.ndarray'>
<class 'list'>
{0: '[0,0,0,0,0,0,0]', 1: '[0,0,0,0,0,0,1]', 2: '[0,0,0,0,0,1,0]', 3: '[0,0,0,0,1,0,0]', 4: '[0,0,0,0,1,0,1]', 5: '[0,0,0,0,1,1,0]', 6: '[0,0,0,1,0,0,0]', 7: '[0,0,0,1,1,0,0]', 8: '[0,1,1,0,0,0,0]', 9: '[0,1,1,1,0,0,0]', 10: '[1,0,0,0,0,0,0]', 11: '[1,0,0,1,0,0,0]', 12: '[1,0,1,0,0,0,0]', 13: '[1,0,1,1,0,0,0]', 14: '[1,1,1,0,0,0,0]', 15: '[1,1,1,1,0,0,0]'}


In [23]:
# 使用方式
# 使用 str.rsplit 分隔字符串，并保留前面的部分
path, _ = label_dir.rsplit('/', 1)

number_of_directories = count_directories(path)
print(f'Number of directories: {number_of_directories}')

# 指定数据集路径和类别数量

dataset_path = path
num_classes = number_of_directories

# 获取每个类别的路径和图像文件名列表
class_paths = []
for i in range(num_classes):
    
    class_path = os.path.join(dataset_path, "{}".format(i))
    class_files = os.listdir(class_path)
#    print(class_files[:10])
    class_paths.append((class_path, class_files))

# 打乱每个类别的图像列表
for class_path, class_files in class_paths:
    random.shuffle(class_files)

# 将数据集分成训练集和测试集
train_file = open(dataset_path + "/train.txt", "w")
test_file = open(dataset_path + "/test.txt", "w")
for i, (class_path, class_files) in enumerate(class_paths):
    num_train = int(len(class_files) * 0.7)
    for j, filename in enumerate(class_files):
        if j < num_train:
            train_file.write("{} {}\n".format(os.path.join(class_path, filename), i))
        else:
            test_file.write("{} {}\n".format(os.path.join(class_path, filename), i))
train_file.close()
test_file.close()

Number of directories: 16


In [24]:
# 检查是否有可用的GPU
if torch.cuda.is_available():
    device = torch.device("mps")
else:
    print("MPS is not available. Using CPU.")
    device = torch.device("cpu")

## resnet
#net = torchvision.models.resnet18(weights='IMAGENET1K_V1')
net = torchvision.models.resnet34(weights='IMAGENET1K_V1')
net.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
net.to(device)  # 将模型移动到CUDA设备上

MPS is not available. Using CPU.


ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [25]:
# 自定义图片图片读取方式
def MyLoader(path):
    return Image.open(path).convert('L')

# def MyLoader(path):
#     return np.load(path, allow_pickle=True)
    
    
class MyDataset (Dataset):
    # 构造函数设置默认参数
    def __init__(self, txt, transform=None, target_transform=None, loader=MyLoader):
        with open(txt, 'r') as fh:
            imgs = []
            for line in fh:
                line = line.strip('\n')  # 移除字符串首尾的换行符
                line = line.rstrip()  # 删除末尾空
                words = line.split( )  # 以空格为分隔符 将字符串分成
                imgs.append((words[0], int(words[1]))) # imgs中包含有图像路径和标签
        self.imgs = imgs
        self.transform = transform
        self.target_transform = target_transform
        self.loader = loader

    def __getitem__(self, index):
        fn, label = self.imgs[index]
        #调用定义的loader方法
        img = self.loader(fn)
        if self.transform is not None:
            img = self.transform(img)
        return img, label

    def __len__(self):
        return len(self.imgs)



root = path + '/'
train_data = MyDataset(txt=root + 'train.txt', transform=transforms.ToTensor())
test_data = MyDataset(txt=root + 'test.txt', transform=transforms.ToTensor())

test_num = len(test_data)

#train_data 和test_data包含多有的训练与测试数据，调用DataLoader批量加载
trainloader = DataLoader(dataset=train_data, batch_size=32, shuffle=True)
testloader = DataLoader(dataset=test_data, batch_size=32, shuffle=False)

print('加载成功！')

import torch.nn.functional as F
from sklearn.model_selection import KFold
import torch.optim as optim

# CrossEntropyLoss就是我们需要的损失函数
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

# Parameters
patience = 50  # for early stopping
n_folds = 10   # for k-fold cross-validation
num_epochs = 200

# K-Fold Cross Validation
# kfold = KFold(n_splits=n_folds, shuffle=True, random_state=5)
kfold = KFold(n_splits=n_folds, shuffle=True)
early_stop_counter = 0
best_loss = float('inf')
best_model_wts = None

for fold, (train_ids, val_ids) in enumerate(kfold.split(trainloader.dataset)):
    print(f"FOLD {fold + 1}/{n_folds}")
    
    # Define your network and optimizer here, they should be re-initialized for each fold
    optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    val_subsampler = torch.utils.data.SubsetRandomSampler(val_ids)
    
    
    trainloader_fold = torch.utils.data.DataLoader(trainloader.dataset, batch_size=32, sampler=train_subsampler)
    valloader_fold = torch.utils.data.DataLoader(trainloader.dataset, batch_size=32, sampler=val_subsampler)
    
    for epoch in range(num_epochs):
        print('-'*30, '\n','epoch', epoch)
        net.train()
        loss100 = 0.0
        correct = 0
        total = 0
        for i, data in enumerate(trainloader_fold):
            # Training code remains largely the same...
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device) # 注意需要复制到GPU
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels.to(device))
            loss.backward()
            optimizer.step()

            loss100 += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        # Validation
        net.eval()
        val_loss = 0.0
        with torch.no_grad():
            for i, data in enumerate(valloader_fold):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = net(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

        avg_val_loss = val_loss / len(valloader_fold)
        print(f"Validation Loss: {avg_val_loss:.4f}")

        # Check for early stopping
        if avg_val_loss < best_loss:
            best_loss = avg_val_loss
            best_model_wts = net.state_dict().copy()  # 更新最佳模型的权重
            early_stop_counter = 0
        else:
            early_stop_counter += 1

        if early_stop_counter >= patience:
            print("Early stopping!")
            break
# 保存最佳模型的权重到文件
model_save_path = path +'/' +'best_model.pth'
torch.save(best_model_wts, model_save_path)
# Finally, you could report the average performance across all folds.
#plt.plot(train_accuracies)
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training Accuracy')
plt.show()

加载成功！
FOLD 1/10
------------------------------ 
 epoch 0


KeyboardInterrupt: 

In [10]:
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import balanced_accuracy_score
predicted_list = []
labels_list = []
# 构造测试的dataloader
dataiter = iter(testloader)
# 预测正确的数量和总数量
correct = 0
total = 0
net.eval()
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        # 预测
        outputs = net(images)
        # 输出概率分布，最大概率的一项作为预测分类
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        # 将预测结果和真实结果添加到列表中
        predicted_list.extend(predicted.cpu().numpy())
        labels_list.extend(labels.cpu().numpy())
        # 计算损失
        loss = criterion(outputs, labels)


confusion_mat = confusion_matrix(labels_list, predicted_list)
print('Accuracy : %f %%' % (
    100 * correct / test_num))
print('Loss : %f %%' % (
    loss.item()))
balanced_accuracy = balanced_accuracy_score(labels_list, predicted_list, adjusted=False)
print('Balanced Accuracy : %f %%' %(100 * balanced_accuracy))
print(confusion_mat)

Accuracy : 73.333333 %
Loss : 3.031840 %
Balanced Accuracy : 47.410714 %
[[ 2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  1  0  0  0  1  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  1  0  0  1  0  0  0  0  1  0  0  0  0  0]
 [ 0  0  0 21  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  2  0  1  2  0  0  0  0  0  1  0  0  0  0  0]
 [ 0  1  1  0  2  1  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 28  2  0  0  0  0  0  0  0  0]
 [ 0  0  0  1  0  0  1 11  0  0  0  0  0  0  1  0]
 [ 0  0  0  0  0  0  0  0  2  0  0  0  0  0  1  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  2  0  2  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  1  1  0  0  1]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0]
 [ 0  0  0  0  0  0  0  1  2  0  0  0  1  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  5]]
